# Saturação do **GRADIENTE**

Geralmente ocorre em funções de ativação não lineares como Sigmoid e Tangente Hiperbólica (tanh), acontece quando os gradientes se tornam muito pequenos, dificultando o aprendizado da rede

✅ Para evitar saturação:

- Prefira ReLU, Leaky ReLU ou ELU em vez de sigmoide/tanh.

- Use Batch Normalization, que mantém os valores das ativações dentro de uma faixa saudável.

# Fuga do **GRADIENTE**

Ocorre quando os gradientes se tornam muito grandes, fazendo com que os pesos explodam e a rede fique instável. Esse problema é comum em redes muito profundas, principalmente com funções de ativação que não possuem limite superior. pode acontecer em **Redes Recorrentes**, **Variantes da ReLU (Leaky ReLU, PReLU, ELU)**, funções que não possuem **limite superior para os valores da ativação**.

✅ Para evitar fuga do gradiente:

- Use Inicialização de Pesos adequada (ex: He Initialization para ReLU).

- Utilize Batch Normalization para manter os gradientes sob controle.

- Gradiente Clipping: Limita o tamanho do gradiente para evitar explosão.

- Regularização L2 (Weight Decay): Penaliza pesos muito grandes.

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# LeakyRelu

Utilizando a função de ativação *RelU* (onde o valor de entrada positivo permanece o mesmo, enquanto uma entrada negativa se torna zero), pode ser que o nosso modelo mate alguns neurônios pois se um neurônio recebe valores negativos continuamente, sempre retornará zero. A *LeakyRelu* visa consertar isso. Em sua função, ao invés de zerar valores de entrada negativa, ela os transforma em um fator α pequeno, geralmente α = 0.01, fazendo assim que em algum momento aquele neurônio "volte a vida", aprendendo novamente

## 📌 Como escolher a função de ativação da última camada?

| **Tipo de Problema**                                          | **Função de Ativação na Saída**         | **Explicação** |
|--------------------------------------------------------------|---------------------------------|--------------|
| **Classificação binária** (2 classes: "sim/não", "gato/cachorro") | `sigmoid`                     | Retorna um valor entre **0 e 1**, interpretado como **probabilidade**. |
| **Classificação multiclasse** (3 ou mais classes, previsão de uma única classe) | `softmax`                     | Converte os logits em **probabilidades normalizadas** (somam 1), útil para **1 classe por amostra**. |
| **Classificação multirrótulo** (cada amostra pode ter várias classes) | `sigmoid`                      | Cada saída pode ser ativada independentemente, pois cada rótulo é tratado separadamente. |
| **Regressão** (previsão de valores contínuos, como temperatura, preço de casa) | `linear` (ou nenhuma ativação) | Permite qualquer valor real como saída, sem restrições. |
| **Regressão para valores positivos** (exemplo: previsão de contagem) | `relu` ou `softplus`          | Garante que a saída seja **sempre positiva**. |


In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), # Tamanho das imagens
    keras.layers.Dense(300, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(alpha=0.2), # pode ser "PReLU()" no lugar da LeakyReLU, mas se quiser usar SELU precisa definir "activation='selu'" e "kernel_initializer='cun_normal'"
    keras.layers.Dense(100, kernel_initializer='he_normal'),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dense(10, activation='softmax') # 10 pois são 10 classes possíveis de saída. (classes das fotos do MNIST)
])

c:\Users\anton\OneDrive\Desktop\RedesNeurais-Basico\environment\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\anton\OneDrive\Desktop\RedesNeurais-Basico\environment\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [6]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3), metrics=['accuracy'])

In [7]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4755 - loss: 1.6578 - val_accuracy: 0.7168 - val_loss: 0.8749
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7209 - loss: 0.8360 - val_accuracy: 0.7734 - val_loss: 0.7050
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7734 - loss: 0.6927 - val_accuracy: 0.8020 - val_loss: 0.6257
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7959 - loss: 0.6230 - val_accuracy: 0.8136 - val_loss: 0.5801
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8076 - loss: 0.5804 - val_accuracy: 0.8222 - val_loss: 0.5515
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8149 - loss: 0.5552 - val_accuracy: 0.8302 - val_loss: 0.5246
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8195 - loss: 0.5360 - val_accuracy: 0.8314 - val_loss: 0.5095
Epoch 8/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8280 - loss: 0.5122 - 

# ❌ Piora no modelo

Por que será que o modelo utilizando o leakyRelU piorou em relação ao que fizemos no arquivo "RedesSequenciais.ipynb" utilizando uma rede sequencial simples?



Fator |	Modelo com LeakyReLU (pior desempenho) |	Modelo com ReLU (melhor desempenho)
|------------------------------------------------------|---------------------------------|--------------|
Função de ativação |	LeakyReLU(α=0.2) (mantém gradientes negativos, pode atrapalhar) |	ReLU (aprendizado mais rápido e eficiente)
Inicialização |	he_normal (pode gerar pesos grandes demais) |	glorot_uniform (mais equilibrado para redes pequenas)
Complexidade |	Mais camadas e inicializações desnecessárias |	Modelo mais simples e eficiente

# 🎲 batch Normalization

Utilizada para **normalizar** as **ativações das camadas intermediárias** de uma rede neural, o **BathNormalization** acelera o treinamento, melhora a performance e reduz a sensibilidade a inicialização dos pesos.

Quando treinamos uma rede neural, as ativações das camadas podem ficar em diferentes escalas, o que pode dificultar a cnovergência durante o treinamento, já que o gradiente pode se tornar muito grande ou muito pequeno. Isso é conhecido como **covariate shift** (Mudança na distribuição das ativações das camadas). O **BatchNormalization** garante que as ativações tenham distribuição com média **0** e desvio padrão **1** durante o treinamento.

In [8]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3) ,metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6035 - loss: 1.1884 - val_accuracy: 0.7980 - val_loss: 0.5868
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7859 - loss: 0.6183 - val_accuracy: 0.8260 - val_loss: 0.5099
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8117 - loss: 0.5461 - val_accuracy: 0.8372 - val_loss: 0.4738
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8198 - loss: 0.5173 - val_accuracy: 0.8464 - val_loss: 0.4516
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8279 - loss: 0.4956 - val_accuracy: 0.8516 - val_loss: 0.4364
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8350 - loss: 0.4687 - val_accuracy: 0.8544 - val_loss: 0.4251
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8355 - loss: 0.4645 - val_accuracy: 0.8582 - val_loss: 0.4155
Epoch 8/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8410 - loss: 0.4557 - 

Mais uma vez, houve uma piora pois o problema é "simples" e não requer um aprendizado tão profundo

# 📥Dropout

Um dos reguladores mais comuns nas redes neurais profunda são as camadas de *Dropout*, as redes neurais mais modernas podem obter um aumento de 1 a 2% de acurácia simplesmente por adicionar camadas de *Dropout* segundo umartigo de 2014 de **Nitish Srivastava**.

Como o Dropout funciona?
Durante o treinamento de uma rede neural, o Dropout "desativa" aleatoriamente um conjunto de neurônios em cada iteração, de maneira que eles não participam da ativação nem do cálculo do gradiente. Esse processo é aplicado de forma estocástica, ou seja, em cada atualização do modelo, diferentes neurônios são desativados.

Por exemplo, se aplicarmos um Dropout de 50%, metade dos neurônios serão desativados aleatoriamente em cada ciclo de treinamento. Isso força a rede a aprender representações mais robustas e a não depender excessivamente de neurônios específicos.

In [11]:
import os
import time
logdir = os.path.join(os.curdir, "ArquivosGeradosPelosCodigos/logs")

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(logdir, run_id)

run_logdir = get_run_logdir()

# Criando TransBoard como callbackk
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [12]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation='softmax')
])

In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3) ,metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[tensorboard_cb])

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.3702 - loss: 1.8105 - val_accuracy: 0.7122 - val_loss: 0.8202
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6537 - loss: 0.9512 - val_accuracy: 0.7596 - val_loss: 0.6885
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7061 - loss: 0.8158 - val_accuracy: 0.7844 - val_loss: 0.6284
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7331 - loss: 0.7476 - val_accuracy: 0.7982 - val_loss: 0.5920
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7455 - loss: 0.7089 - val_accuracy: 0.8066 - val_loss: 0.5654
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7564 - loss: 0.6778 - val_accuracy: 0.8112 - val_loss: 0.5450
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7679 - loss: 0.6490 - val_accuracy: 0.8192 - val_loss: 0.5305
Epoch 8/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7720 - loss: 0.6376 - 

No treinamento de redes neurais, momentum e decay são técnicas usadas para melhorar a eficiência e estabilidade da descida do gradiente. Elas ajudam a evitar problemas como mínimos locais e oscilações, além de acelerar a convergência.

# 🎢 Momentum
O momentum é uma técnica usada para suavizar a atualização dos pesos durante o treinamento. Ele funciona como uma espécie de "memória" dos passos anteriores, ajudando a rede a continuar na mesma direção e reduzindo oscilações.

Fórmula do Gradient Descent com Momentum:

$$
v_t = \beta v_{t-1} + (1 - \beta) \nabla J(\theta_t)
$$

$$
\theta_t = \theta_{t-1} - \alpha v_t
$$


 Benefícios do **Momentum**:

✅ Acelera a convergência em direções consistentes.

✅ Reduz oscilações em direções instáveis.

✅ Evita mínimos locais rasos ao manter a "inércia" da atualização.

📌 **Exemplo intuitivo**: Imagine uma bola rolando por um vale; o momentum faz com que ela continue se movendo mesmo se o gradiente ficar próximo de zero por um tempo.

#  Decay (Decaimento da Taxa de Aprendizado)
O decay (ou "learning rate decay") controla a redução gradual da taxa de aprendizado (𝛼) ao longo do treinamento. Isso ajuda a rede a fazer ajustes mais finos nos pesos à medida que se aproxima da solução ótima.

Benefícios do **Decay**:

✅ Permite passos grandes no início (exploração) e pequenos no fim (convergência precisa).

✅ Evita que a rede fique "presa" em mínimos locais subótimos.

✅ Aumenta a estabilidade do treinamento.

📌 **Exemplo intuitivo**: No início, você dá passos largos para encontrar uma solução geral, mas conforme se aproxima da resposta ideal, começa a dar passos menores para refiná-la.

# 🎲 **Resumo**
Conceito	| Objetivo	| Como funciona
-------|----------|---------
Momentum	| Suaviza e acelera a otimização	| Mantém "memória" das atualizações passadas para reduzir oscilações e acelerar a convergência
Decay	| Ajusta a taxa de aprendizado ao longo do tempo	| Começa com um learning rate alto e o reduz gradualmente para evitar passos muito grandes no final


Muitos  otimizadores modernos, como Adam, RMSprop e SGD com Momentum, já integram esses conceitos para melhorar o desempenho do treinamento.

In [17]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9, decay=0.01) ,metrics=['accuracy'])

# Momentum = O valor 0.9 indica que 90% do gradiente anterior é mantido para a próxima atualização, e apenas 10% vem do gradiente atual. Ajuda a rede a seguir na direção certa, evitando oscilações.
# Decay = 0.01 → Reduz gradativamente a taxa de aprendizado para melhorar a precisão no final.

model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6204 - loss: 1.0807 - val_accuracy: 0.8242 - val_loss: 0.5056
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7884 - loss: 0.5923 - val_accuracy: 0.8402 - val_loss: 0.4622
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8059 - loss: 0.5380 - val_accuracy: 0.8496 - val_loss: 0.4326
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8149 - loss: 0.5090 - val_accuracy: 0.8532 - val_loss: 0.4186
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8200 - loss: 0.4990 - val_accuracy: 0.8576 - val_loss: 0.4085
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8256 - loss: 0.4857 - val_accuracy: 0.8518 - val_loss: 0.4139
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8280 - loss: 0.4772 - val_accuracy: 0.8594 - val_loss: 0.3957
Epoch 8/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8282 - loss: 0.4697 - 